In [1]:
import pandas as pd
import numpy as np

In [2]:
# Rutas de los archivos de Excel
ruta_unidades_sm = r'C:\Users\aramosm\Documents\MIX\EXPORTACIONES\UNIDADES_SM.xlsx'
ruta_ventas_sm = r'C:\Users\aramosm\Documents\MIX\EXPORTACIONES\VENTAS_SM.xlsx'
ruta_producto_t = r'C:\Users\aramosm\Documents\MIX\EXPORTACIONES\PRODUCTO_T.xlsx'

In [3]:
# Cargar los archivos de Excel en DataFrames
df_unidades_sm = pd.read_excel(ruta_unidades_sm)
df_ventas_sm = pd.read_excel(ruta_ventas_sm)
df_producto_t = pd.read_excel(ruta_producto_t)

In [4]:
# Generar el ID en df_unidades_sm concatenando las columnas 4query_UMES_SubEntidad, 4query_UMES_Empleado y 4query_UMES_Nom_Producto
df_unidades_sm['ID'] = df_unidades_sm['4query_UMES_SubEntidad'].astype(str) + '_' + df_unidades_sm['4query_UMES_Empleado'].astype(str) + '_' + df_unidades_sm['4query_UMES_Nom_Producto'].astype(str)

In [5]:
# Generar el ID en df_ventas_sm concatenando las columnas 4query_VMES_SubEntidad, 4query_VMES_Empleado y 4query_VMES_Nom_Producto
df_ventas_sm['ID'] = df_ventas_sm['4query_VMES_SubEntidad'].astype(str) + '_' + df_ventas_sm['4query_VMES_Empleado'].astype(str) + '_' + df_ventas_sm['4query_VMES_Nom_Producto'].astype(str)

In [6]:
# Cambiar el nombre de la columna 'v1nprd' a 'SKU'
#df_producto_t = df_producto_t.rename(columns={'v1nprd': 'SKU'})
df_ventas_sm = df_ventas_sm.rename(columns={
    '4query_VMES_SubEntidad':'SubEntidad', 
     '4query_VMES_Empleado': 'Empleado',
    '4query_VMES_Nom_Producto': 'SKU'
})

In [7]:
# Convertir la columna 'ID' y 'SKU' a tipo de datos compatible
df_unidades_sm['ID'] = df_unidades_sm['ID'].astype(str)
df_ventas_sm['ID'] = df_ventas_sm['ID'].astype(str)
df_producto_t['SKU'] = df_producto_t['SKU'].astype(str)

In [8]:
# Combinar los DataFrames de UNIDADES_SM y VENTAS_SM utilizando una combinación externa izquierda
df_union_uv = pd.merge(df_unidades_sm, df_ventas_sm, on='ID', how='right')

In [9]:
column_order = list(df_ventas_sm.columns) + list(df_unidades_sm.columns)
df_union_uv = df_union_uv[column_order]

In [10]:
# Combinar el resultado anterior con el DataFrame de PRODUCTO_T utilizando una combinación interna
df_resultado = pd.merge(df_union_uv, df_producto_t, on='SKU', how='left')

In [11]:
column_order2 = list(df_producto_t.columns) + list(df_union_uv.columns)
df_resultado = df_resultado[column_order2]

In [12]:
# Eliminar las columnas especificadas de df_resultado
columnas_eliminar = ['4query_UMES_SubEntidad', '4query_UMES_Empleado', '4query_UMES_Nom_Producto',
                     '4query_USEM_SubEntidad', '4query_USEM_Empleado', '4query_USEM_Nom_Producto',
                     '4query_VSEM_SubEntidad', '4query_VSEM_Empleado', '4query_VSEM_Nom_Producto','ID']
df_resultado = df_resultado.drop(columnas_eliminar, axis=1)


In [13]:
# Selecciona las columnas numéricas
columnas_numericas = df_resultado.select_dtypes(include='number').columns

# Guarda las columnas de texto sin modificar
columnas_texto = df_resultado.select_dtypes(include='object')

# Reemplaza los valores mayores a 100 con NaN en las columnas numéricas
df_resultado[columnas_numericas] = df_resultado[columnas_numericas].where(df_resultado[columnas_numericas] >= 100, np.nan)

# Combina las columnas numéricas filtradas con las columnas de texto
df_completo = pd.concat([df_resultado[columnas_numericas], columnas_texto], axis=1)


In [14]:
# Exportar el resultado a un nuevo archivo Excel
ruta_salida = r'C:\Users\aramosm\Documents\MIX\EXPORTACIONES\MIX_VU.xlsx'
ruta_salida100 = r'C:\Users\aramosm\Documents\MIX\EXPORTACIONES\MIX_VU100.xlsx'
df_resultado.to_excel(ruta_salida, index=False)
df_completo.to_excel(ruta_salida100, index=False)

In [15]:
print("El archivo MIX_VU.xlsx ha sido creado con éxito.")

El archivo MIX_VU.xlsx ha sido creado con éxito.
